# Kepler Framework, Emulate v15

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [7]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'


In [8]:
tpath_data17 = 'output/emulation/data17_table_v15.h5'
tpath_data18 = 'output/emulation/data18_table_v15.h5'


## Setup Chains:

In [9]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v15_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v15_ivarloose" , L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v15_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v15_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v15"          , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v15_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v15_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v15_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v15_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v15_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [10]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [11]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:28<00:00, 22.01s/it]


In [12]:
data17_df.shape

(43311283, 112)

### Validate:

In [13]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v15.root')

2022-03-24 10:47:09,653 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v15.root


In [14]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v15'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.55s/it]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.3924  | 29929991/32047563 |
|   L2   | 93.1531  | 29853301/32047563 |
| EFCalo | 91.8442  | 29433815/32047563 |
|  HLT   | 89.9546  | 28828261/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:25<00:00, 17.07s/it]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2254  | 21450972/22765602 |
|   L2   | 94.1099  | 21424690/22765602 |
| EFCalo | 93.1851  | 21214138/22765602 |
|  HLT   | 87.0126  | 19808946/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 96.8624  | 1324870/1367786 |
|   L2   | 96.7718  | 1323631/1367786 |
| EFCalo | 94.6772  | 1294982/1367786 |
|  HLT   | 91.4192  | 1250419/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.54it/s]

HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 96.3843  | 58352/60541  |
|   L2   | 96.3182  | 58312/60541  |
| EFCalo | 95.0199  | 57526/60541  |
|  HLT   | 93.2509  | 56455/60541  |
+--------+----------+--------------+


In [15]:
eff_data17_Zee.save()

In [16]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v15.root')

2022-03-24 10:51:30,872 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v15.root


In [18]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', pidname='!el_lhvloose', reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v15'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:21<00:00,  4.35s/it]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  0.8733  |  90384/10349992  |
|   L2   |  0.8313  |  86038/10349992  |
| EFCalo |  0.7233  |  74863/10349992  |
|  HLT   |  0.0860  |  8900/10349992   |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.92s/it]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.2947  |  27272/9254300  |
|   L2   |  0.2707  |  25049/9254300  |
| EFCalo |  0.2193  |  20294/9254300  |
|  HLT   |  0.0009  |    86/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.30s/it]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.2074  |  16288/7852181  |
|   L2   |  0.1787  |  14028/7852181  |
| EFCalo |  0.1158  |   9091/7852181  |
|  HLT   |  0.0007  |    55/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.44s/it]

HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  0.5021  |  28290/5634041  |
|   L2   |  0.4475  |  25212/5634041  |
| EFCalo |  0.3148  |  17738/5634041  |
|  HLT   |  0.0158  |   889/5634041   |
+--------+----------+-----------------+


In [19]:
eff_data17_JF17.save()

In [20]:
del data17_df

## Emulate 2018:

In [21]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [22]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:23<00:00, 20.86s/it]


In [23]:
data18_df.shape

(41873973, 112)

In [24]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v15.root')

2022-03-24 11:00:35,085 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v15.root


In [25]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v15'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:57<00:00, 23.49s/it]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.4351  | 30478996/32620495 |
|   L2   | 93.2008  | 30402555/32620495 |
| EFCalo | 91.8691  | 29968162/32620495 |
|  HLT   | 90.4510  | 29505571/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:23<00:00, 16.63s/it]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.1899  | 21959279/23313826 |
|   L2   | 94.0940  | 21936900/23313826 |
| EFCalo | 93.2945  | 21750521/23313826 |
|  HLT   | 87.5786  | 20417927/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.19s/it]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 96.8991  | 1338830/1381675 |
|   L2   | 96.8100  | 1337599/1381675 |
| EFCalo | 94.7188  | 1308706/1381675 |
|  HLT   | 91.7537  | 1267738/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.45it/s]

HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 96.3950  | 58212/60389  |
|   L2   | 96.3205  | 58167/60389  |
| EFCalo | 95.1713  | 57473/60389  |
|  HLT   | 93.5518  | 56495/60389  |
+--------+----------+--------------+


In [26]:
eff_data18_Zee.save()

In [27]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v15.root')

2022-03-24 11:04:56,085 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v15.root


In [28]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v15_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v15'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.51s/it]


HLT_e17_lhvloose_nod0_ringer_v15_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  1.2037  |  101916/8467098 |
|   L2   |  1.1636  |  98519/8467098  |
| EFCalo |  1.0572  |  89513/8467098  |
|  HLT   |  0.4313  |  36516/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.16s/it]


HLT_e28_lhtight_nod0_ringer_v15_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.5186  |  39565/7628804  |
|   L2   |  0.4960  |  37840/7628804  |
| EFCalo |  0.4460  |  34022/7628804  |
|  HLT   |  0.0280  |   2136/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.70s/it]


HLT_e60_lhmedium_nod0_ringer_v15_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.3958  |  25974/6561985  |
|   L2   |  0.3685  |  24183/6561985  |
| EFCalo |  0.3087  |  20255/6561985  |
|  HLT   |  0.0314  |   2060/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.03s/it]

HLT_e140_lhloose_nod0_ringer_v15
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  0.8230  |  39309/4776518  |
|   L2   |  0.7703  |  36793/4776518  |
| EFCalo |  0.6425  |  30687/4776518  |
|  HLT   |  0.2255  |  10769/4776518  |
+--------+----------+-----------------+


In [29]:
eff_data18_JF17.save()